In [11]:
!ls data/gear_images/hardshell_jackets/

10116634x1038116_zm.jpeg  10300351x1076923_zm.jpeg  10331928x1043700_zm.jpeg
10116634x1068361_zm.jpeg  10300351x1108394_zm.jpeg  10331928x1050984_zm.jpeg
10116634x1072858_zm.jpeg  10300351x1128450_zm.jpeg  10331928x1072105_zm.jpeg
101634.jpeg		  10300351x1149781_zm.jpeg  10331928x1084272_zm.jpeg
10184183x1012360_zm.jpeg  10300494x1012905_zm.jpeg  10331928x1084490_zm.jpeg
10184183x1012905_zm.jpeg  10300494x1094207_zm.jpeg  10331929x1050984_zm.jpeg
10184183x1068533_zm.jpeg  10300494x1108394_zm.jpeg  10331929x1125962_zm.jpeg
10184183x1140536_zm.jpeg  10300494x1128450_zm.jpeg  10331929x1130244_zm.jpeg
10184183x1140544_zm.jpeg  10300494x1149691_zm.jpeg  10331929x1150402_zm.jpeg
10184184x1035919_zm.jpeg  10300495x1012905_zm.jpeg  10331929x1150915_zm.jpeg
10184184x1036924_zm.jpeg  10300495x1108394_zm.jpeg  10332124x1121963_zm.jpeg
10184184x1061637_zm.jpeg  10300495x1140836_zm.jpeg  10332124x1131592_zm.jpeg
10184184x1120173_zm.jpeg  10300495x1149691_zm.jpeg  10332246x1046388_zm.jpeg
10184184x1

In [12]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

In [16]:
# Replace with a valid key
training_key = "195f2c767e6c4ffc85fe2ac15ac5b320"
prediction_key = "b67dcfc490e849499c8342bff96a5be9"

trainer = training_api.TrainingApi(training_key)

In [17]:
project = trainer.create_project("openhack18")

In [31]:
hardshellJackets_tag = trainer.create_tag(project.id, "hardshell-jackets")
insulatedJackets_tag = trainer.create_tag(project.id, "insulated-jackets")

In [24]:
import os

In [39]:
base_image_url = "data/gear_images/"

print ("Adding images...")
for image_url in os.listdir(path=base_image_url+"hardshell_jackets/"):
    with open(base_image_url+"hardshell_jackets/"+image_url, mode='rb') as img_data:
        trainer.create_images_from_data(project.id, img_data, [ hardshellJackets_tag.id ])

Adding images...


In [40]:
for image_url in os.listdir(path=base_image_url+"insulated_jackets/"):
    with open(base_image_url+"insulated_jackets/"+image_url, mode='rb') as img_data:
        trainer.create_images_from_data(project.id, img_data, [ insulatedJackets_tag.id ])

### TRAIN

In [41]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


### Predictions

In [42]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

In [46]:
# Now there is a trained endpoint that can be used to make a prediction

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)


# insulated test
test_img_url = "https://image.sportsmansguide.com/adimgs/l/6/673942i3_ts.jpg"
results = predictor.predict_image_url(project.id, iteration.id, url=test_img_url)

# Alternatively, if the images were on disk in a folder called Images alongside the sample.py, then
# they can be added by using the following.
#
# Open the sample image and get back the prediction results.
# with open("Images\\test\\test_image.jpg", mode="rb") as test_data:
#     results = predictor.predict_image(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	insulated-jackets: 100.00%
	hardshell-jackets: 0.00%


In [47]:
# Now there is a trained endpoint that can be used to make a prediction

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)


# hardshell test
test_img_url = "https://cdn.hiconsumption.com/wp-content/uploads/2015/12/Westcomb-Shift-LT-Hoodie-Jacket.jpeg"
results = predictor.predict_image_url(project.id, iteration.id, url=test_img_url)

# Alternatively, if the images were on disk in a folder called Images alongside the sample.py, then
# they can be added by using the following.
#
# Open the sample image and get back the prediction results.
# with open("Images\\test\\test_image.jpg", mode="rb") as test_data:
#     results = predictor.predict_image(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))


	hardshell-jackets: 93.92%
	insulated-jackets: 1.53%
